<a href="https://colab.research.google.com/github/DaraSamii/DataDays2021/blob/main/Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Login

In [ ]:
from google.colab import drive

drive.mount("/content/drive",force_remount=True)

%cd /content/drive/MyDrive/DataDays2021/

pwd = %pwd
pwd

Mounted at /content/drive
/content/drive/MyDrive/DataDays2021


'/content/drive/MyDrive/DataDays2021'

# installign Packages

In [ ]:
!pip install "dask[complete]"
!pip install -Uqq fastai
!pip install parsivar

     |████████████████████████████████| 118 kB 10.8 MB/s 
     |████████████████████████████████| 778 kB 42.8 MB/s 
     |████████████████████████████████| 776 kB 44.6 MB/s 
     |████████████████████████████████| 769 kB 43.1 MB/s 
     |████████████████████████████████| 766 kB 43.9 MB/s 
     |████████████████████████████████| 1.0 MB 40.9 MB/s 
     |████████████████████████████████| 722 kB 40.3 MB/s 
     |████████████████████████████████| 722 kB 45.4 MB/s 
     |████████████████████████████████| 715 kB 35.5 MB/s 
     |████████████████████████████████| 705 kB 18.1 MB/s 
     |████████████████████████████████| 699 kB 39.1 MB/s 
     |████████████████████████████████| 696 kB 42.6 MB/s 
     |████████████████████████████████| 684 kB 41.0 MB/s 
     |████████████████████████████████| 679 kB 41.7 MB/s 
     |████████████████████████████████| 675 kB 31.5 MB/s 
     |████████████████████████████████| 675 kB 37.5 MB/s 
     |████████████████████████████████| 672 kB 39.9 MB/s 
     |████████

In [ ]:
import dask.dataframe as dd
import pandas as pd
import os
from fastai.text.all import *
from fastai.tabular.all import *
import pickle

from torch.utils.data import Dataset
from tqdm import tqdm
import torch as T
from fastai.data.core import DataLoaders
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader

from helper import utils

from tqdm import tqdm

In [ ]:
data_folder = os.path.join(pwd,"data")
final_clicked = os.path.join(data_folder,"final","final_clicked.csv")
final_products = os.path.join(data_folder,"final","final_products.csv")
mix_model_path = os.path.join(data_folder,"models","mix_model","models","mix_model.pth")
category_classifier_path = os.path.join(data_folder,"models","category_classifier","models","category_classifier.pth")
vocab_path = os.path.join(data_folder,"models","category_classifier","vocab")
category_list_path = os.path.join(data_folder,"models","category_classifier","catagory_list")
queries_test_offline_normalized_path = os.path.join(data_folder, "queries_test_offline_normalized.json")
queries_test_offline_path = os.path.join(data_folder, "queries_test_offline.json")
test_queries_floder_path = os.path.join(data_folder,"final","results.csv")

In [ ]:
vocab = pickle.load(open(vocab_path,'rb'))
vocab

(#2) [['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', '<', '>', '2', '1', 'startquery', 'endquery', 'startcategory', 'endcategory', 'و', 'گوشی', 'سایر', 'لوازم', '،', 'سامسونگ', 'بازی', 'جانبی', 'شیائومی', 'خودرو', 'ساعت', 'ماشین', 'مردانه', 'کیف', 'برقی', 'زنانه', 'کفش', 'لپ\u200cتاپ', 'تبلت', 'ابزار', 'برندهای', 'لباس', 'بادی', 'ساز', 'هدفون', 'کامپیوتر', 'اسباب', 'تجهیزات', 'کاور', 'تلویزیون', 'تفنگ', 'اپل', 'هوشمند', 'هدست', 'کنسول', 'یخچال', 'خواب', 'شارژی', 'کولر', 'هندزفری', 'کودک', 'ای', 'دستگاه', 'صندل', 'مودم', 'موبایل', 'هارد', 'سونی', 'آیفون', 'قاب', 'باتری', 'اسپیکر', 'ماسک', 'فریزر', 'بند', 'ایسوس', 'کیبورد', 'پی', 'چراغ', 'ساید', 'صندلی', 'ماوس', 'دوربین', 'ps4', 'دوچرخه', 'موتور', 'دخترانه', 'آشپزخانه', 'ظروف', 'ال', 'کارت', 'گازی', 'فلش', 'میز', 'poco', 'تاپ', '10', 'سرویس', 'روغن', 'پز', 'pro', 'جی', 'نوت', 'گل', 'کشتی', 'ورزشی', 'قطار', 'اسلایم', 'تخت', 'شارژر', 'اصلاح', 'note', 'اسکوتر', 'اسکیت', 'انواع', 'ایکس', 'آرایش', 'مچ', 'قطع

In [ ]:
category_list = pickle.load(open(category_list_path,'rb'))
category_list[:10]

['سایر لوازم تزیینی خودرو',
 'شلوار و دامن زنانه',
 'کفش و صندل زنانه',
 'ساندویچ ساز و وافل ساز',
 'سایر لوازم و تجهیزات زیبایی و ارایش',
 'سایر برندهای پاوربانک',
 'عینک و لوازم جانبی',
 'محافظ صفحه سایر برندهای گوشی',
 'ماشین اصلاح و ریش تراش',
 'سایر لوازم سفر و گردشگری']

In [ ]:
class nlp_pipeline:
    def __init__(self, vocab,):
        self.vocab = vocab
        self.tok = SpacyTokenizer(lang='fa')
        self.num = Numericalize(vocab=self.vocab)

    def encode(self,x):
        x = utils._normalize_text(x)
        x = tokenize1(x, self.tok)
        x = self.num.encodes(x)
        return x

    def decode(self,x):
        x = self.num.decodes(x)
        x = " ".join(x)
        return x

In [ ]:
def make_category_ds(query, category_list):
    df =  pd.DataFrame({"category_name":category_list})
    df["query"] = utils._normalize_text(query)
    return df

In [ ]:
class categoryDataSet(Dataset):
    def __init__(self, df, query_column_name, category_column_name, vocab ,target_column_name = None,test=False):
        self.df = df.reset_index(drop=True)
        self.query_column_name = query_column_name
        self.category_column_name = category_column_name
        self.test = test
        if test == False:
            self.target_column_name = target_column_name

        self.vocab= vocab

        self.nlp_pipeline = nlp_pipeline(vocab=self.vocab)

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, i):
        x1 = self.df[self.query_column_name][i]
        x1_numericalized = self.nlp_pipeline.encode(x1)

        x2 = self.df[self.category_column_name][i]
        x2_numericalized = self.nlp_pipeline.encode(x2)

        if self.test == False:
            y = self.df[self.target_column_name][i]
            return (x1_numericalized,x2_numericalized), y

        else:
            return (x1_numericalized,x2_numericalized)

In [ ]:
def my_collate_category(batch):
    x1,x2 = list(zip(*batch))


    return (nn.utils.rnn.pad_sequence(x1).T.cuda(),  nn.utils.rnn.pad_sequence(x2).T.cuda())

In [ ]:
class CatageoryClassifier(nn.Module):
    def __init__(self, vocab_sz, awd_config, lin_ftrs,joint_layers,joint_drop =0.5):
        super(CatageoryClassifier, self).__init__()
        self.vocab_sz = vocab_sz
        self.awd_config = awd_config 
        self.lin_ftrs = lin_ftrs

        self.joint_drop = joint_drop

        self.awd1 = get_text_classifier(AWD_LSTM,vocab_sz=vocab_sz, n_class=2, config=awd_config,lin_ftrs=lin_ftrs)
        self.awd1[-1].layers = self.awd1[-1].layers[:-1]

        self.awd2 = get_text_classifier(AWD_LSTM,vocab_sz=vocab_sz, n_class=2, config=awd_config,lin_ftrs=lin_ftrs)
        self.awd2[-1].layers = self.awd2[-1].layers[:-1]

        self.joint_layers = [lin_ftrs[-1]*2] + joint_layers + [2]
        linBins = []
        for i in range(0,len(self.joint_layers)-1):
            linBins.append(LinBnDrop(self.joint_layers[i],self.joint_layers[i+1]))

        self.LinBins = nn.Sequential(*linBins)
        self.Softmax = nn.Softmax(dim=1)


    def forward(self,x):
        x1 = self.awd1(x[0])[0]
        x2 = self.awd2(x[1])[0]

        X = T.cat([x1,x2],dim=1)
        X = self.LinBins(X)
        return self.Softmax(X)

    def reset(self):
        self.awd1.reset()
        self.awd2.reset()

In [ ]:
awd_conf = {'bidir': True,
            'emb_sz': 1000,
            'embed_p': 0.05,
            'hidden_p': 0.3,
            'input_p': 0.4,
            'n_hid': 1800,
            'n_layers': 5,
            'output_p': 0.4,
            'pad_token': 1,
            'weight_p': 0.5}

In [ ]:
nn_model  = CatageoryClassifier(len(vocab[0]),awd_config=awd_conf,lin_ftrs=[1000],joint_layers=[500])

In [ ]:
state_dict = T.load(open(category_classifier_path,'rb'),map_location=T.device('cpu'))
nn_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
def Similarity(sent1, sent2):
    try:
        count = 0
        for word1 in sent1.split(" "):
            for word2 in sent2.split(" "):
                if word1 == word2:
                    count+=1
        return count
    except:
        return 0

In [ ]:
def category_classifer(query, vocab, category_list, model, bs=180):
    df = make_category_ds(query, category_list)

    df["similarity"] = df.apply(lambda x: Similarity(x["category_name"],x["query"]),axis=1)
    
    df.sort_values("similarity",ascending=False,inplace=True)

    if df["similarity"].max() == 0:
        test_ds = categoryDataSet(df,"query","category_name",vocab=vocab,test=True)
    else:
        df = df[0:100] 
        test_ds = categoryDataSet(df,"query","category_name",vocab=vocab,test=True)

    test_dl = DataLoader(dataset=test_ds, batch_size=bs, shuffle=False, collate_fn=my_collate_category)

    model = model.cuda()
    Y = []
    model.reset()
    for batch in tqdm(iter(test_dl)):
        batch_reslut = model(batch)
        Y.append(batch_reslut.detach().cpu())

    model.reset()
    
    Ys =T.cat(Y)
    
    df["False"] = Ys[:,0].numpy()
    df["True"] = Ys[:,1].numpy()

    

    df["score"] = df.apply( lambda x: (x["similarity"] + 1) * x["True"],axis=1)

    return df.sort_values(["score","True","False"],ascending=[False,False,True])

In [ ]:
category_classifer("تلویزیون 40 ایتچ", vocab[0], category_list,nn_model,bs=50).head(10)

100%|██████████| 2/2 [00:00<00:00,  4.39it/s]


,category_name,query,similarity,False,True,score
69,تلویزیون سامسونگ,تلویزیون 40 ایتچ,1,1.748477e-25,1.000000,2.000000
726,سایر برند تلویزیون,تلویزیون 40 ایتچ,1,3.222410e-19,1.000000,2.000000
629,تلویزیون سونی,تلویزیون 40 ایتچ,1,1.345351e-12,1.000000,2.000000
455,تلویزیون اسنوا,تلویزیون 40 ایتچ,1,2.637388e-12,1.000000,2.000000
209,سایر تلویزیون و لوازم جانبی,تلویزیون 40 ایتچ,1,1.143444e-08,1.000000,2.000000
1412,تلویزیون,تلویزیون 40 ایتچ,1,3.355507e-06,0.999997,1.999993
637,تلویزیون هایسنس,تلویزیون 40 ایتچ,1,6.043648e-06,0.999994,1.999988
961,کنسول بازی مایکروسافت xbox,تلویزیون 40 ایتچ,0,1.776986e-34,1.000000,1.000000
923,نمایشگر ایسوس,تلویزیون 40 ایتچ,0,3.563499e-14,1.000000,1.000000
931,لوازم جانبی ابسرد کن,تلویزیون 40 ایتچ,0,8.647007e-11,1.000000,1.000000


In [ ]:
category_classifer("موبایل سامسونگ", vocab[0], category_list,nn_model,bs=50).head(10)

100%|██████████| 2/2 [00:00<00:00,  4.80it/s]


,category_name,query,similarity,False,True,score
350,تبلت سامسونگ,موبایل سامسونگ,1,1.349305e-17,1.000000,2.00000
1073,لپ تاپ سامسونگ,موبایل سامسونگ,1,2.612290e-16,1.000000,2.00000
142,گوشی سامسونگ,موبایل سامسونگ,1,4.807710e-15,1.000000,2.00000
69,تلویزیون سامسونگ,موبایل سامسونگ,1,4.117890e-14,1.000000,2.00000
312,سایر لوازم جانبی موبایل و تبلت,موبایل سامسونگ,1,7.541625e-14,1.000000,2.00000
323,دسته بازی مخصوص موبایل,موبایل سامسونگ,1,1.258031e-11,1.000000,2.00000
261,هدفون و هدست سامسونگ,موبایل سامسونگ,1,3.152921e-10,1.000000,2.00000
872,پاوربانک سامسونگ,موبایل سامسونگ,1,4.670732e-10,1.000000,2.00000
90,سایر برندهای گوشی موبایل,موبایل سامسونگ,1,1.069268e-09,1.000000,2.00000
1225,فلش مموری سامسونگ,موبایل سامسونگ,1,8.470693e-05,0.999915,1.99983


In [ ]:
category_classifer("گوشی هوآوی", vocab[0], category_list,nn_model,bs=50).head(10)

100%|██████████| 2/2 [00:00<00:00,  4.74it/s]


,category_name,query,similarity,False,True,score
1433,کاور گوشی هواوی,گوشی هواوی,2,1.230090e-09,1.00000,3.00000
485,گوشی هواوی,گوشی هواوی,2,1.289880e-03,0.99871,2.99613
204,پایه نگهدارنده گوشی و تبلت,گوشی هواوی,1,3.919120e-16,1.00000,2.00000
332,گوشی موتورولا,گوشی هواوی,1,9.055989e-15,1.00000,2.00000
41,گوشی شیایومی,گوشی هواوی,1,1.851630e-14,1.00000,2.00000
625,گوشی نوکیا,گوشی هواوی,1,1.432632e-13,1.00000,2.00000
59,کیف و کاور گوشی و تبلت,گوشی هواوی,1,2.395801e-13,1.00000,2.00000
720,گوشی سونی,گوشی هواوی,1,5.985493e-12,1.00000,2.00000
448,گوشی انر,گوشی هواوی,1,1.534926e-11,1.00000,2.00000
944,باتری گوشی شیایومی,گوشی هواوی,1,3.030202e-11,1.00000,2.00000


In [ ]:
nan_df = category_classifer("تلویزیون", vocab[0], category_list,nn_model,bs=50).head(10)
nan_df

100%|██████████| 2/2 [00:00<00:00,  5.39it/s]


,category_name,query,similarity,False,True,score
69,تلویزیون سامسونگ,تلویزیون,1,4.334778e-31,1.000000,2.000000
629,تلویزیون سونی,تلویزیون,1,2.832253e-16,1.000000,2.000000
455,تلویزیون اسنوا,تلویزیون,1,1.079520e-12,1.000000,2.000000
209,سایر تلویزیون و لوازم جانبی,تلویزیون,1,1.953436e-08,1.000000,2.000000
1162,تلویزیون فیلیپس,تلویزیون,1,1.104353e-05,0.999989,1.999978
726,سایر برند تلویزیون,تلویزیون,1,3.481781e-05,0.999965,1.999930
1026,تلویزیون مجیک,تلویزیون,1,1.409094e-03,0.998591,1.997182
829,تلویزیون پاناسونیک,تلویزیون,1,1.704378e-03,0.998296,1.996591
655,تلویزیون دوو,تلویزیون,1,4.679795e-02,0.953202,1.906404
1323,تلویزیون توشیبا,تلویزیون,1,2.190648e-01,0.780935,1.561870


In [ ]:
nan_cat = list(nan_df["category_name"][0:5])
nan_cat

['تلویزیون سامسونگ',
 'تلویزیون سونی',
 'تلویزیون اسنوا',
 'سایر تلویزیون و لوازم جانبی',
 'تلویزیون فیلیپس']

# mix_dl maker

In [ ]:
products = dd.read_csv(final_products)
products.head()

,Unnamed: 0,Unnamed: 0.1,category_name,product_name_normalized,sellers_count,availabilty_ratio,mean_all_price,max_all_price,min_all_price,std_all_price,skew_all_price,mean_available_price,max_available_price,min_available_price,std_available_price,skew_available_price,COUNT(clicked_merged),PERCENT_TRUE(clicked_merged.is_clicked),DAY(first_added_date),DAY(last_added_date),HOUR(first_added_date),HOUR(last_added_date),IS_WEEKEND(first_added_date),IS_WEEKEND(last_added_date),MINUTE(first_added_date),MINUTE(last_added_date),MONTH(first_added_date),MONTH(last_added_date),NUM_WORDS(product_name_normalized),WEEKDAY(first_added_date),WEEKDAY(last_added_date),YEAR(first_added_date),YEAR(last_added_date),PERCENT_TRUE(clicked_merged.IS_WEEKEND(datetime)),_id
0,0,0,ویدیو پروژکتور اپتما,optoma دیتا خانگی 1920 x 1080 3 d صفحه پروژکتور hd 142 x رزولوشن lumens خانگی optoma video theater dlp home hd 142 x projector hd optoma اپتما 1080 p مدل 30 اصلی اوپتوما ویدیو 3 d سینمای,6,0.166667,1.625193e+07,31852600.0,11500000.0,7.104350e+06,1.655126,3.185260e+07,31852600.0,31852600.0,0.000000,0.000000,0.0,0.000000,12.0,25.0,7.0,20.0,False,True,2.0,8.0,9.0,1.0,38,1.0,5.0,2017.0,2020.0,0.000000,yxwrs
1,1,1,تقویت کننده مژه و ابرو,کننده mascara ژل gel and essence mascara essence اسنس دهنده lash and brow گرم lash essence اصل mascara and 9 ابرو gel mascara and sweet و miss اصلی لیفت مژه تقویت gel حالت essence brow حجم brow essence اورجینال lash,33,0.696970,6.681758e+04,125000.0,2430.0,2.521815e+04,-0.605363,7.282826e+04,125000.0,21500.0,20905.381178,-0.133438,12.0,0.166667,27.0,10.0,20.0,12.0,False,False,21.0,18.0,2.0,3.0,40,2.0,2.0,2019.0,2021.0,0.333333,mbeby
2,2,2,مام و اسپری,tuning v 8 stick ایت وزن دیودرانت استیک تعریق rexona گرم رکسونا 40 مردانه ضد deodorant rexona men مدل دیودورانت وی for,5,0.000000,3.120000e+04,60000.0,0.0,1.907249e+04,-0.185370,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,30.0,14.0,19.0,15.0,False,False,5.0,16.0,4.0,5.0,22,1.0,3.0,2019.0,2020.0,0.000000,yovgc
3,3,3,مام و اسپری,women رول stick 40 ml 48 h وزن دیودرانت algodon زنانه motion میلی صابونی cotton استیک تعریق rexona با کاتون رکسونا dry گرم 40 مام لیتر ضد ضدتعریق algodon dry deodorant کاتن algodon sense بدن rexona درحجم مدل 40 درای for cotton میل حجم الگودون rexona,16,0.750000,5.606750e+04,99500.0,40000.0,1.464371e+04,1.745850,5.916500e+04,99500.0,45000.0,15565.639777,1.448900,0.0,0.000000,16.0,3.0,13.0,23.0,False,False,52.0,39.0,1.0,2.0,46,2.0,2.0,2019.0,2021.0,0.000000,yokff
4,4,4,گوشی هواوی,ascend huawei موبایل 550 مدل گوشی 4 g وای y 550 ascend هواوی اسند هواوی huawei,5,0.000000,7.100000e+04,355000.0,0.0,1.420000e+05,1.500000,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,24.0,12.0,19.0,10.0,False,False,33.0,26.0,4.0,3.0,16,0.0,0.0,2017.0,2018.0,0.000000,uduyq


In [ ]:
products_pd = products.compute()

In [ ]:
products_pd["category_name"] = products_pd["category_name"].apply(lambda x: utils._normalize_text(x))

In [ ]:
products_pd.to_csv(final_products)

In [ ]:
def make_clicked_dl(query, categories,products1):
    df = products1[products1["category_name"].isin(list(map(lambda x: utils._normalize_text(x),categories)))]

    df = df.drop("Unnamed: 0",axis=1)
    df = df.rename(columns={k:("products." + k) for k in df.columns})

    df["rank"] = 0
    df["page"] = 0
    df["NUM_WORDS(raw_query)"] = len(query.split(" "))
    df["raw_query"] = utils._normalize_text(query)

    return df

In [ ]:
make_clicked_dl("test",['سایر برندهای لپ تاپ و نوت بوک', 'شارژر شیایومی'],products_pd)

(549, 34)
(549, 33)
(549, 37)


,products.category_name,products.product_name_normalized,products.sellers_count,products.availabilty_ratio,products.mean_all_price,products.max_all_price,products.min_all_price,products.std_all_price,products.skew_all_price,products.mean_available_price,products.max_available_price,products.min_available_price,products.std_available_price,products.skew_available_price,products.COUNT(clicked_merged),products.PERCENT_TRUE(clicked_merged.is_clicked),products.DAY(first_added_date),products.DAY(last_added_date),products.HOUR(first_added_date),products.HOUR(last_added_date),products.IS_WEEKEND(first_added_date),products.IS_WEEKEND(last_added_date),products.MINUTE(first_added_date),products.MINUTE(last_added_date),products.MONTH(first_added_date),products.MONTH(last_added_date),products.NUM_WORDS(product_name_normalized),products.WEEKDAY(first_added_date),products.WEEKDAY(last_added_date),products.YEAR(first_added_date),products.YEAR(last_added_date),products.PERCENT_TRUE(clicked_merged.IS_WEEKEND(datetime)),products._id,rank,page,NUM_WORDS(raw_query),raw_query
12075,سایر برندهای لپ تاپ و نوت بوک,g sync 2080 10 r 0 gaming intel 16 gb بوک i 7 9700 k 10 nvme 8 gb area intel gamer nvidia and quot fhd 9 th gen ram i 7 9700 k 51 m home genel aw sshd hd ram laptop 144 hz 1920 and quot full eye gb 1 tb xpc 2 x 256 gb laptop windows ssd 16 256 geforce 17 3 tobii 2070 1080 144 hz نوت x vr rtx 2070 core,3,1.000000,1.153928e+08,120347200.0,109949400.0,4.258944e+06,-0.170712,1.153928e+08,120347200.0,109949400.0,4.258944e+06,-0.170712,0.0,0.000000,27.0,25.0,18.0,11.0,False,False,37.0,1.0,3.0,8.0,77,4.0,1.0,2020.0,2020.0,0.0,mmejj,0,0,1,test
12087,سایر برندهای لپ تاپ و نوت بوک,ویندوز type c نمایشگر 10 1 n 4100 صفحه chuwi پردازنده and 39 and 39 2 11 6 با لمسی رایانه ubook ای در usb هسته گیگابایت ssd لوحی 256 اینتل نمایش 3 0 چهار 1080 p 8 رم,1,1.000000,1.255600e+07,12556000.0,12556000.0,0.000000e+00,0.000000,1.255600e+07,12556000.0,12556000.0,0.000000e+00,0.000000,0.0,0.000000,27.0,27.0,18.0,18.0,False,False,47.0,47.0,3.0,3.0,39,4.0,4.0,2020.0,2020.0,0.0,ujnso,0,0,1,test
17374,سایر برندهای لپ تاپ و نوت بوک,chromebook pixelbook 64 gb google سیاه 8 gb فقط go m 3,1,1.000000,2.295220e+07,22952200.0,22952200.0,0.000000e+00,0.000000,2.295220e+07,22952200.0,22952200.0,0.000000e+00,0.000000,6.0,0.000000,3.0,3.0,19.0,19.0,False,False,12.0,12.0,4.0,4.0,12,4.0,4.0,2020.0,2020.0,1.0,mqbdw,0,0,1,test
17375,سایر برندهای لپ تاپ و نوت بوک,128 chromebook gb pixelbook google i 5 8 gb black فقط go,1,1.000000,2.920140e+07,29201400.0,29201400.0,0.000000e+00,0.000000,2.920140e+07,29201400.0,29201400.0,0.000000e+00,0.000000,0.0,0.000000,3.0,3.0,19.0,19.0,False,False,12.0,12.0,4.0,4.0,12,4.0,4.0,2020.0,2020.0,0.0,mqbds,0,0,1,test
40410,سایر برندهای لپ تاپ و نوت بوک,هارد 16 gb nvme intel nvidia گرافیک quadro پردازنده لپ i 7 9750 h تاپ t 10 ddr 4 conceptd 5 512 gb مدل کارت ایسر 15 6 ssd رم,1,1.000000,5.816280e+07,58162800.0,58162800.0,0.000000e+00,0.000000,5.816280e+07,58162800.0,58162800.0,0.000000e+00,0.000000,0.0,0.000000,22.0,22.0,19.0,19.0,False,False,54.0,54.0,4.0,4.0,30,2.0,2.0,2020.0,2020.0,0.0,mcivx,0,0,1,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177844,شارژر شیایومی,xiaomi دیواری شیایومی اداپتور فست 18 w 20 travel 18 charger سریع واتی mdy 08 eh شارژر وات شارژ مدل کلگی اورجینال,9,0.444444,1.635556e+05,250000.0,0.0,7.268977e+04,-0.896732,2.257500e+05,250000.0,185000.0,2.467159e+04,-0.849802,0.0,0.000000,15.0,10.0,1.0,22.0,True,False,48.0,53.0,2.0,5.0,22,5.0,0.0,2020.0,2021.0,0.0,kulwk,0,0,1,test
177845,شارژر شیایومی,فست zmi 65 واتی شارژر شیایومی ha 712,1,0.000000,4.200000e+05,420000.0,420000.0,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaN,3.0,0.333333,13.0,13.0,2.0,2.0,False,False,39.0,39.0,2.0,2.0,8,3.0,3.0,2020.0,2020.0,0.0,uirxu,0,0,1,test
187698,شارژر شیایومی,فست adapter اصلی xiaomi سریع power charger دیواری usb کابل 18 w شارژر و شیایومی mdy 08 ei micro,

In [ ]:
cont_cols = ["rank",
             "NUM_WORDS(raw_query)",
             "page",
             "products.sellers_count",
             "products.availabilty_ratio",
             "products.mean_all_price",
             "products.max_all_price",
             "products.min_all_price",
             "products.std_all_price",
             "products.skew_all_price",
             "products.mean_available_price",
             "products.max_available_price",
             "products.min_available_price",
             "products.std_available_price",
             "products.skew_available_price",
             "products.COUNT(clicked_merged)",
             "products.PERCENT_TRUE(clicked_merged.is_clicked)",
             "products.NUM_WORDS(product_name_normalized)"]

cat_cols = ['products.DAY(first_added_date)',
            'products.DAY(last_added_date)',
            'products.HOUR(first_added_date)',
            'products.HOUR(last_added_date)',
            'products.IS_WEEKEND(first_added_date)',
            'products.IS_WEEKEND(last_added_date)',
            'products.MINUTE(first_added_date)',
            'products.MINUTE(last_added_date)',
            'products.MONTH(first_added_date)',
            'products.MONTH(last_added_date)',
            'products.WEEKDAY(first_added_date)',
            'products.WEEKDAY(last_added_date)',
            'products.YEAR(first_added_date)',
            'products.YEAR(last_added_date)']

text_col = ['raw_query',
            'products.category_name',
            'products.product_name_normalized',]

In [ ]:
class MixDataSet(Dataset):
    def __init__(self, df, cat_col_names, cont_col_names, query_col_name, category_col_name, product_col_name, vocab,target_col_name=None, test=False,normalize=True,):
        self.df = df.reset_index(drop=True).copy()
        self.cat_col_names = cat_col_names
        self.cont_col_names = cont_col_names
        self.query_col_name = query_col_name
        self.category_col_name = category_col_name
        self.product_col_name = product_col_name
        self.target_col_name = target_col_name


        self.test = test

        self.normalize = normalize

        self.vocab = vocab
        self.nlp_pipeline = nlp_pipeline(vocab=self.vocab)

        if self.test == False:
            self.target_col_name = target_col_name


    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i):
        ndf = self.df.iloc[i]
        ndf = ndf.fillna(0.0,inplace=False)

        if self.normalize == True:
            if type(ndf) == pd.core.series.Series:
             for col in ndf.index:
                if "price" in col:
                    ndf[col] = np.log10(ndf[col]/1000 + 1)
                elif "NUM" in col or "COUNT" in col:
                    ndf[col] = np.log10(ndf[col] + 1)
           
            elif type(ndf) == pd.core.frame.DataFrame:
             for col in ndf.columns:
                if "price" in col:
                    ndf[col] = np.log10(ndf[col]/1000 + 1)
                elif "NUM" in col or "COUNT" in col:
                    ndf[col] = np.log10(ndf[col] + 1)

        for col in self.cat_col_names:
            if "YEAR" in col:
                ndf[col] = ndf[col] - 2017

            
        cat = ndf[self.cat_col_names].values

        cont = ndf[self.cont_col_names].values

        query = self.nlp_pipeline.encode(ndf[self.query_col_name])
        category = self.nlp_pipeline.encode(ndf[self.category_col_name])
        product = self.nlp_pipeline.encode(ndf[self.product_col_name])
        
        if self.test == False:
            target = ndf[self.target_col_name]
            return (T.tensor(cat.astype(np.int32)),T.tensor(cont.astype(np.float32)),query,category,product), T.tensor(target)
        else:
            return (T.tensor(cat.astype(np.int32)),T.tensor(cont.astype(np.float32)),query,category,product)

In [ ]:
def mix_my_collate(batch):
    x1,x2,x3,x4,x5 = list(zip(*batch))

    return (T.stack(x1).cuda(),
            T.stack(x2).cuda(),
            nn.utils.rnn.pad_sequence(x3).T.cuda(),
            nn.utils.rnn.pad_sequence(x4).T.cuda(),
            nn.utils.rnn.pad_sequence(x5).T.cuda())

In [ ]:
class MixModel(nn.Module):
    def __init__(self, embed_sz, n_cont, layers, awd_config, vocab_sz, lin_ftrs,joint_layers):
        super(MixModel, self).__init__()

        self.embed_sz  = embed_sz
        self.n_cont = n_cont
        self.layers = layers
        self.awd_config = awd_config
        self.joint_layers = joint_layers

        self.tab_model = TabularModel(emb_szs=self.embed_sz, n_cont=n_cont, out_sz=2,layers=layers,)
        self.tab_model.layers = self.tab_model.layers[:-1]

        self.awd1 = get_text_classifier(AWD_LSTM,vocab_sz=vocab_sz, n_class=2, config=awd_config,lin_ftrs=lin_ftrs)
        self.awd1[-1].layers = self.awd1[-1].layers[:-1]

        self.awd2 = get_text_classifier(AWD_LSTM,vocab_sz=vocab_sz, n_class=2, config=awd_config,lin_ftrs=lin_ftrs)
        self.awd2[-1].layers = self.awd2[-1].layers[:-1]

        self.awd3 = get_text_classifier(AWD_LSTM,vocab_sz=vocab_sz, n_class=2, config=awd_config,lin_ftrs=lin_ftrs)
        self.awd3[-1].layers = self.awd3[-1].layers[:-1]

        self.joint_layers = [lin_ftrs[-1]*3 + layers[-1]] + joint_layers + [2]
        linBins = []
        for i in range(0,len(self.joint_layers)-1):
            linBins.append(LinBnDrop(self.joint_layers[i],self.joint_layers[i+1]))

        self.LinBins = nn.Sequential(*linBins)
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, x):
        xtab = self.tab_model(nn.functional.relu(x[0]),x[1])
        xnlp1 = self.awd1(x[2])[0]
        xnlp2 = self.awd2(x[3])[0]
        xnlp3 = self.awd3(x[4])[0]

        X = T.cat([xtab, xnlp1, xnlp2, xnlp3],dim=1)
        X = self.LinBins(X)
        return self.Softmax(X)

    def reset(self):
        self.awd1.reset()
        self.awd2.reset()
        self.awd3.reset()

In [ ]:
emb_sz = [
          (32, 20),
          (32, 20),
          (24, 20),
          (24, 20),
          (3, 10),
          (3, 10),
          (61, 30),
          (61, 30),
          (13, 10),
          (13, 10),
          (8, 10),
          (8, 10),
          (6, 20),
          (6, 20)
]

awd_conf = {'bidir': True,
            'emb_sz': 1000,
            'embed_p': 0.05,
            'hidden_p': 0.3,
            'input_p': 0.4,
            'n_hid': 1000,
            'n_layers': 3,
            'output_p': 0.4,
            'pad_token': 1,
            'weight_p': 0.5}

In [ ]:
mix_model = MixModel(embed_sz = emb_sz, n_cont= len(cont_cols), layers=[200], awd_config=awd_conf, vocab_sz=len(vocab[0]), lin_ftrs=[500],joint_layers=[500])

In [ ]:
state_dict = T.load(open(mix_model_path,'rb'),map_location=T.device('cpu'))
mix_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
def clicked_classifier(query, categories_list,products1,vocab,mix_model,bs=100):
    pdf = make_clicked_dl(query,categories_list,products1)
    pdf["similarity"] = 0
    pdf["similarity"] = pdf.apply(lambda x: Similarity(str(x["products.product_name_normalized"]),str(x["raw_query"])),axis=1)
    pdf.sort_values(["similarity","products.sellers_count"],ascending=[False,False],inplace=True,axis=0)

    if pdf["similarity"].max() == 0:
        pdf = pdf[0:1000]
        mix_test_ds1 = MixDataSet(pdf,cat_cols,cont_cols,"raw_query","products.category_name","products.product_name_normalized",vocab=vocab,test=True)
    else:
        pdf = pdf[0:500] 
        mix_test_ds1 = MixDataSet(pdf,cat_cols,cont_cols,"raw_query","products.category_name","products.product_name_normalized",vocab=vocab,test=True)
    
    mix_test_dl1 = DataLoader(dataset=mix_test_ds1, batch_size=bs, shuffle=False, collate_fn=mix_my_collate)


    
    mix_model = mix_model.cuda()
    mix_model.reset()
    
    Y = []
    for batch in tqdm(iter(mix_test_dl1)):
        res = mix_model(batch)
        Y.append(res.detach().cpu())
    
    mix_model.reset()
    Ys = T.cat(Y)

    pdf["False"] = Ys[:,0].numpy()
    pdf["True"] = Ys[:,1].numpy()

  

    pdf["score"] = pdf.apply( lambda x: (x["similarity"] + 1) * x["True"],axis=1)

    return pdf.sort_values(["score","True","False"],ascending=[False,False,True])[text_col + ["True","False",
                                                                "similarity","score",
                                                                'products.sellers_count',
                                                                'products.availabilty_ratio',
                                                                'products.COUNT(clicked_merged)',
                                                                'products._id']]

In [ ]:
clicked_classifier("تلویزیون",nan_cat,products_pd,vocab[0],bs=100,mix_model=mix_model)

100%|██████████| 5/5 [00:04<00:00,  1.24it/s]


,raw_query,products.category_name,products.product_name_normalized,True,False,similarity,score,products.sellers_count,products.availabilty_ratio,products.COUNT(clicked_merged),products._id
207358,تلویزیون,تلویزیون سونی,sony lcd سونی 55 x 80 g اندرویدی tv x 80 g x 80 g ال 4 k size inch کیفیت model عالی 55 x 80 g تلویزیون با kd 55 x 80 g 55 تلویزیون 55 x 80 g ultrahd hd اندروید ای android inch دی و مدل sony سی ultra 4 k سایز اینچ سونی هوشمند smart led 55 اینچ تصویر uhd smart اینچ 55 55 sony,1.000000e+00,2.833921e-13,2,3.000000e+00,53,0.490566,293.0,mmjld
165524,تلویزیون,تلویزیون سونی,sony inch سری tv 55 x 8500 g ال 4 k inch کیفیت مدل 55 x 8500 g عالی الترا sony 55 x 8500 g تلویزیون با kd 55 x 8500 g sony 55 تلویزیون 55 55 x 8500 g ultrahd 55 x 8500 g اندروید اچ ای android جی دی x 8500 g و مدل sony سی 4 k سایز اینچ سونی هوشمند led smart 55 اینچ sony مدل uhd smart تصویر اینچ 55 55,1.000000e+00,2.191934e-11,2,3.000000e+00,65,0.523077,244.0,mmlcd
179223,تلویزیون,تلویزیون سونی,bravia sony اسمارت سونی tv x x 9500 ال 4 k inch 4 k hdr 55 x 9500 g kd 55 x 9500 g تلویزیون براویا 55 x 9500 g تلویزیون 55 ultrahd hd 9500 اندروید مشخصات ای android 55 x 9500 g 55 x 9500 g g دی x 9500 g مدل sony ultra 55 x 9500 g 4 k سایز اینچ هوشمند سونی led smart 55 اینچ smart اینچ 55 55,1.000000e+00,2.054453e-10,2,3.000000e+00,39,0.512821,231.0,mmjlz
152796,تلویزیون,تلویزیون سامسونگ,55 اسمارت دی 55 سری tv ای مد 55 n 6900 15816 ال inch samsung 55 n 6900 inch کیفیت samsung تلویزیون با full تلویزیون hd ای 6 flat full 55 n 6900 دی مدل کد دی fhd سایز هوشمند سامسونگ led smart تصویر 55 اینچ smart اینچ 55 سایز 55 n 6900,1.000000e+00,8.180614e-10,2,3.000000e+00,44,0.340909,370.0,mmlnc
171624,تلویزیون,تلویزیون اسنوا,snowa sld 32 sa 220 مدل sld 32 sa 220 تلویزیون full tv 32 ال 32 sa 220 sa 220 inch snowa تلویزیون hd اچ ای اسنوا sld 32 sa 260 دی 32 مدل سایز led اینچ سایز 32,1.000000e+00,1.484440e-09,2,3.000000e+00,24,0.125000,126.0,mmvly
...,...,...,...,...,...,...,...,...,...,...,...
45554,تلویزیون,تلویزیون سامسونگ,q 60 rak 65 سامسونگ اینچ 65 q 60 rak تلویزیون مدل,1.225218e-19,1.000000e+00,1,2.450437e-19,2,0.500000,0.0,ygbyp
159876,تلویزیون,تلویزیون سامسونگ,49 49 منحنی m 6500 ال samsung 49 m 6500 تلویزیون 49 m 6500 hd ای full دی مدل سایز سامسونگ هوشمند led اینچ 49 m 6500,8.506445e-21,1.000000e+00,1,1.701289e-20,11,0.272727,0.0,mybcb
154869,تلویزیون,تلویزیون سامسونگ,49 سامسونگ اینچ 49 ku 7350 تلویزیون مدل,1.781090e-21,1.000000e+00,1,3.562180e-21,3,0.333333,0.0,mvlgn
199797,تلویزیون,تلویزیون سامسونگ,49 49 منحنی 4 k 49 nu 7300 سری uhd tv samsung صفحه ال 4 k inch samsung الترا 4 k تلویزیون hd اچ ای 49 nu 7300 4 k دی nu 7300 49 nu 7300 مدل ultra سایز سامسونگ هوشمند led فور کی اینچ,6.076474e-22,1.000000e+00,1,1.215295e-21,22,0.227273,0.0,ufawh


In [ ]:
test_list = []
for i in utils.read_json(queries_test_offline_normalized_path):
    test_list.append(i)
test_list = list(set(test_list))

38724it [00:00, 224513.47it/s]


In [ ]:
len(test_list)

28696

In [ ]:
class Recommander:
    def __init__(self, products, vocab, category_list, category_model, mix_model):
        self.products = products
        self.vocab = vocab
        self.category_list = category_list
        self.category_model = category_model
        self.mix_model = mix_model
 
 
    def recomend(self, query):
        T.cuda.empty_cache()
        cdf = category_classifer(query, self.vocab, self.category_list, self.category_model,bs=100)
        cdf.reset_index(drop=True)

        best_candid = cdf[cdf["score"] > cdf["score"].max()/2].shape[0]
        if best_candid < 5:
            best_candid = 5
        elif best_candid > 100:
            best_candid=100

        candid_list = list(cdf.iloc[0:best_candid]["category_name"].values)

        T.cuda.empty_cache()
        pdf = clicked_classifier(query,candid_list,self.products,self.vocab,bs=100,mix_model=self.mix_model)

        return list(pdf["products._id"][0:10].values)

In [ ]:
rec = Recommander(products_pd, vocab[0], category_list,nn_model, mix_model)

In [ ]:
test_list[2]

'کولر گازی سامسونگ ar 25 nsp'

In [ ]:
a = test_list[2]
qwer = rec.recomend(a)
print(a)

(1453, 2)


100%|██████████| 5/5 [00:06<00:00,  1.25s/it]

کولر گازی سامسونگ ar 25 nsp


In [ ]:
products_pd[products_pd["_id"].isin(qwer)]

,Unnamed: 0,category_name,product_name_normalized,sellers_count,availabilty_ratio,mean_all_price,max_all_price,min_all_price,std_all_price,skew_all_price,mean_available_price,max_available_price,min_available_price,std_available_price,skew_available_price,COUNT(clicked_merged),PERCENT_TRUE(clicked_merged.is_clicked),DAY(first_added_date),DAY(last_added_date),HOUR(first_added_date),HOUR(last_added_date),IS_WEEKEND(first_added_date),IS_WEEKEND(last_added_date),MINUTE(first_added_date),MINUTE(last_added_date),MONTH(first_added_date),MONTH(last_added_date),NUM_WORDS(product_name_normalized),WEEKDAY(first_added_date),WEEKDAY(last_added_date),YEAR(first_added_date),YEAR(last_added_date),PERCENT_TRUE(clicked_merged.IS_WEEKEND(datetime)),_id
5148,5148,کولر گازی,سرد دیواری conditioner 30 مکس samsung اسپیلت ar 30 kpfu max گرم کولر سامسونگ ar 30 kpfu ar 30 kpfu max و سری max گازی مدل سامسونگ max air اسپلیت,6,0.000000,1.061917e+07,27750000.0,0.0,9.001894e+06,0.795621,NaN,NaN,NaN,NaN,NaN,21.0,0.000000,26.0,16.0,1.0,11.0,False,False,9.0,50.0,4.0,12.0,30,2.0,2.0,2017.0,2020.0,0.190476,mtfnq
46210,3725,کولر گازی,ar 13 kcfs سرد دیواری سری good conditioner samsung کولر 130 good ar 13 jcfs گازی مدل سرمایشی گود سامسونگ 130 کولرگازی air اسپلیت ar 13 jcfs kcfs,7,0.000000,1.257029e+07,27750000.0,5644000.0,6.741304e+06,1.416939,NaN,NaN,NaN,NaN,NaN,4.0,0.250000,4.0,17.0,17.0,9.0,False,False,6.0,26.0,5.0,12.0,28,3.0,3.0,2017.0,2020.0,0.000000,ypmrt
75572,4050,کولر گازی,سرد ar 30 kcfu max دیواری سری conditioner 30 model conditioner مکس samsung اسپیلت سامسونگ ar 30 kcfu کولر max 30 max گازی گازی samsung مدل air سامسونگ ar 30 kcfu max کولرگازی air اسپلیت split,16,0.437500,2.173712e+07,31000000.0,0.0,1.036944e+07,-1.222281,2.871714e+07,31000000.0,25500000.0,1.776407e+06,-0.611495,184.0,0.054348,3.0,10.0,13.0,16.0,True,False,31.0,48.0,6.0,2.0,36,5.0,2.0,2017.0,2021.0,0.217391,mmnib
198626,1891,کولر گازی,سامسونگ ar 19 ks سرد good 1 inverter دیواری سری good one 190 conditioner good 1 conditione samsung inverter اسپیلت 190 گرم کولر ar 19 ms ar 19 ks ar 19 ks ms و گازی مدل اینورتر series سامسونگ 180 گودوان good 1 کولرگازی air اسپلیت,13,0.230769,1.487338e+07,23850000.0,0.0,7.463761e+06,-0.948278,2.084833e+07,22145000.0,20200000.0,9.168818e+05,0.707107,50.0,0.120000,13.0,1.0,12.0,8.0,False,False,57.0,41.0,11.0,12.0,46,1.0,1.0,2018.0,2020.0,0.200000,mmxdo
70838,8531,کولر گازی,سرد 120 ar 13 ks سری دیواری conditioner ar 13 ks ms good 1 samsung inverter samsung ar 13 ks گرم کولر 130 ar 13 js ar 13 ks هزار اینورترمدل و گازی مدل اینورتر series وان گود سامسونگ گودوان good 1 ar 13 ks 130 کولرگازی air اسپلیت,10,0.200000,1.523100e+07,27750000.0,0.0,7.912775e+06,-0.369744,2.069000e+07,20980000.0,20400000.0,2.900000e+05,0.000000,18.0,0.000000,13.0,17.0,12.0,9.0,False,False,57.0,26.0,11.0,12.0,49,1.0,3.0,2018.0,2020.0,0.055556,mmoyt
82005,8027,کولر گازی,250 ar 25 kpfs mqfs سردوگرم سری conditioner good ar 25 mqfs 240 samsung سرد کولر good 240 و گازی مدل series ar 25 kpfs گود سامسونگ گرم کولرگازی air اسپلیت,6,0.000000,1.754900e+07,23550000.0,10100000.0,4.291996e+06,-0.295127,NaN,NaN,NaN,NaN,NaN,18.0,0.055556,3.0,20.0,13.0,11.0,False,True,53.0,18.0,5.0,9.0,31,2.0,6.0,2017.0,2020.0,0.111111,mgjap
107455,495,کولر گازی,250 کولرسامسونگ boracay سرد سردوگرم سری دیواری conditioner s inverter بوراکای بروکای ar 25 msfhe samsung inverter گرم کولر boracay inverter ar 25 msfh 240 boracay براکای و گازی مدل اینورتر series سامسونگ s inverter s کولرگازی air اسپلیت,22,0.409091,1.830409e+07,28050000.0,0.0,8.694934e+06,-0.650030,2.255400e+07,28050000.0,11500000.0,6.305672e+06,-0.947509,429.0,0.100233,12.0,1.0,6.0,18.0,False,False,30.0,25.0,3.0,4.0,39,0.0,3.0,2018.0,2021.0,0.261072,mmoay
123797,6977,کولر گازی,250 ar 25 kcfu سرد 240 ar 25 kcfu max مکس سامسونگ samsung دیواری conditioner اسپیلت max کولر اسپلیت گازی ar 25 kcfu air مدل,7,0.000000,1.134900e+07,27750000.0,0.0,9.392506e+06,0.327631,NaN,NaN,NaN,NaN,NaN,26.0,0.038462,28.0,16.0,8.0,11.0,False,False,47.0,50.0,6.0,12.0,25,2.0,2.0,2017.0,2020.0,

In [ ]:
results = pd.read_csv(test_queries_floder_path,index_col=0)
results

,queries,results
0,گوشت کوب برقی گاوسونیک,"['uqrdn', 'yvvhc', 'ykytp', 'milfh', 'mfynt', 'udush', 'yfcdg', 'muhss', 'kcudv', 'mcufp']"
1,اسپرسوساز دلونگی 850,"['mmpoc', 'mytio', 'mmpjp', 'mbfqc', 'mmppj', 'mykls', 'mbfvf', 'mbfqv', 'mmpwr', 'mmpdi']"
2,ناخن مسنویی,"['mnmzv', 'mncte', 'mcuui', 'yapyf', 'ucpdf', 'yuobj', 'mbsym', 'mbywr', 'mnmrq', 'uynpi']"
3,تخت معاینه زنان,"['kxmzf', 'mueel', 'ydokk', 'matyr', 'mbfdn', 'mkmea', 'muxvd', 'matvh', 'mgqkq', 'ygfoi']"
4,شیامی نوت 11,"['yngkg', 'mbrxw', 'mqklt', 'kgfmx', 'kqcgq', 'kmbcj', 'upouu', 'kmbcl', 'ukmpk', 'yudss']"
...,...,...
28691,دلر شارژی ronix,N
28692,گوشواره نقره,N
28693,پک هدیه زنانه,N
28694,ماشین اصلاح خطزن,N


In [ ]:
for i in range(results.shape[0]):
    if results.iloc[i]["results"] != "N":
        pass
    else:
        results.at[i,"results"] = rec.recomend(results.at[i,"queries"])
        T.cuda.empty_cache()
        
        print(i ,"/ 28696")
        if i%5 == 0:
            results.to_csv(test_queries_floder_path) 
            print("Saved")

results.to_csv(test_queries_floder_path) 

# upload results

In [ ]:
from google.colab import drive

drive.mount("/content/drive",force_remount=True)

%cd /content/drive/MyDrive/DataDays2021/

pwd = %pwd
pwd

Mounted at /content/drive
/content/drive/MyDrive/DataDays2021


'/content/drive/MyDrive/DataDays2021'

In [ ]:
!pip install parsivar

     |████████████████████████████████| 36.2 MB 62 kB/s 
     |████████████████████████████████| 1.5 MB 66.1 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492971 sha256=0ce611ae07e2129daf40384bc3b060db585316e37c6f8af2b5cf8c6174f30be0
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449920 sha256=3e6d0d44bed22b7805f72177084938b48400b2f5099c09d8d714319fc37d4d66
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import os
import pandas as pd
import numpy as np
from helper import utils
from tqdm import tqdm

In [ ]:
data_folder = os.path.join(pwd,"data")
final_clicked = os.path.join(data_folder,"final","final_clicked.csv")
final_products = os.path.join(data_folder,"final","final_products.csv")
mix_model_path = os.path.join(data_folder,"models","mix_model","models","mix_model.pth")
category_classifier_path = os.path.join(data_folder,"models","category_classifier","models","category_classifier.pth")
vocab_path = os.path.join(data_folder,"models","category_classifier","vocab")
category_list_path = os.path.join(data_folder,"models","category_classifier","catagory_list")
queries_test_offline_normalized_path = os.path.join(data_folder, "queries_test_offline_normalized.json")
queries_test_offline_path = os.path.join(data_folder, "queries_test_offline.json")
test_queries_floder_path = os.path.join(data_folder,"final","results.csv")

In [ ]:
results = pd.read_csv(test_queries_floder_path,index_col=0)
results

,queries,results
0,گوشت کوب برقی گاوسونیک,"['uqrdn', 'yvvhc', 'ykytp', 'milfh', 'mfynt', ..."
1,اسپرسوساز دلونگی 850,"['mmpoc', 'mytio', 'mmpjp', 'mbfqc', 'mmppj', ..."
2,ناخن مسنویی,"['mnmzv', 'mncte', 'mcuui', 'yapyf', 'ucpdf', ..."
3,تخت معاینه زنان,"['kxmzf', 'mueel', 'ydokk', 'matyr', 'mbfdn', ..."
4,شیامی نوت 11,"['yngkg', 'mbrxw', 'mqklt', 'kgfmx', 'kqcgq', ..."
...,...,...
28691,دلر شارژی ronix,N
28692,گوشواره نقره,N
28693,پک هدیه زنانه,N
28694,ماشین اصلاح خطزن,N


In [ ]:
last_test_list = []
for i in utils.read_json(queries_test_offline_path):
    last_test_list.append(i)
len(last_test_list)

38724it [00:00, 42789.17it/s]


38724

In [ ]:
not_normalized = []
for i in utils.read_json('/content/drive/MyDrive/DataDays2021/data/final/queries_test_offline.json'):
    not_normalized.append(i)
len(not_normalized)

38724it [00:00, 39880.10it/s]


38724

In [ ]:
not_normalized[1]

'قهوه سازDelongi'

In [ ]:
output = pd.DataFrame({"normalized_query":last_test_list,
                       "raw_query":not_normalized})

In [ ]:
output

,normalized_query,raw_query
0,تلویزیون 40 ایتچ,تلویزیون 40 ایتچ
1,قهوه ساز delongi,قهوه سازDelongi
2,بند فلزی mi band 5,بند فلزی mi band 5
3,قالی زمردمشهد,قالی زمردمشهد
4,پوکو x 3 pro,پوکوx3 pro
...,...,...
38719,نوت 10,نوت ۱۰+
38720,اسباب بازی بتمت,اسباب‌بازی‌بتمت
38721,کراسفیت,کراسفیت
38722,mate 30 pro,mate30pro


In [ ]:

def finder(query,i):
    try:
        x= str(results[results["queries"] == query]["results"].values[0])
        if x == 'N':
            return 'N'

        x = x.replace("[","").replace("'","").replace("]","").replace(" ","").split(",")[i]
        return x
    except:
        print(results[results["queries"] == query]["results"].values[0])

In [ ]:
for i in tqdm(range(0,10)):
    output[i] = output.apply(lambda x: finder(x["normalized_query"], i),axis=1)

100%|██████████| 1/1 [01:20<00:00, 80.52s/it]


In [ ]:
output[output[1] !="N"]

,normalized_query,raw_query,1,2,3,4,5,6,7,8,9,0
31,چسب ترمیم شیشه,چسب ترمیم شیشه,mdkmv,mzdjo,mszdh,keaxl,ybmrb,yykqm,mdeow,msiub,uqbsi,kgucu
35,اسباب بازز,اسباب بازز,mvsbc,ybelv,mbixl,mehlu,mmark,mdvac,macom,macoz,uprsc,mbcyn
96,مورم ایرانسل,مورم ایرانسل,uktaj,mbbqo,yjkpe,mxamd,myatq,uqmqz,umsdb,yybfh,mtvbq,mbapz
112,عروسک بند انگشتی,عروسک بند انگشتی,myhjd,mmtpp,mmute,mmuav,mmyby,mmftz,mmuxg,muqog,mmach,mmmpf
129,نوار led,نوار LED,uajzr,mpelf,mdppz,mxkks,msxob,mfnii,mziab,mpeld,mfuae,bicij
...,...,...,...,...,...,...,...,...,...,...,...,...
38629,کفش ایمنی طبی,کفش ایمنی طبی,meogg,mbnrk,ydpwj,mdsmv,umxwo,mpydi,mjymm,mywkn,yojek,myevc
38637,نوکیا 5 4,نوکیا5.4,myzhn,mmmis,mmmic,mmyxy,mmkmv,mmmir,mmyon,myfiw,mmdue,mmmyt
38640,ریسه وچراغ تزیینی,ریسه وچراغ تزئینی,yityi,mfbzv,yitmh,yitbn,ubtdd,yityx,khvao,yitbg,yitbt,mifsw
38655,بووم,بووم,mutco,mtgla,kqtdn,merkb,mzvwy,ytyoj,mmbum,mtrga,mmypz,mcnrd


In [ ]:
output[['raw_query', 0, 1, 2, 3,4,5,6,7,8,9]]

,raw_query,0,1,2,3,4,5,6,7,8,9
0,تلویزیون 40 ایتچ,N,N,N,N,N,N,N,N,N,N
1,قهوه سازDelongi,N,N,N,N,N,N,N,N,N,N
2,بند فلزی mi band 5,N,N,N,N,N,N,N,N,N,N
3,قالی زمردمشهد,N,N,N,N,N,N,N,N,N,N
4,پوکوx3 pro,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...
38719,نوت ۱۰+,N,N,N,N,N,N,N,N,N,N
38720,اسباب‌بازی‌بتمت,N,N,N,N,N,N,N,N,N,N
38721,کراسفیت,N,N,N,N,N,N,N,N,N,N
38722,mate30pro,N,N,N,N,N,N,N,N,N,N


In [ ]:
output_path = os.path.join(data_folder,"final","output.csv")

In [ ]:
output.drop("normalized_query",axis=1).to_csv(output_path,header=False, index=False)

In [ ]:
products_pd[products_pd["_id"] == "mncte"]

NameError: ignored